# Batch Normalization

- a.k.a. BatchNorm, BN
- [Paper](https://arxiv.org/abs/1502.03167): Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift
- Sergey Ioffe et al. (Google), 2015

## TL;DR

- 딥러닝 모델의 layer마다 입력을 정규분포로 변환

그러면?

- LR을 좀 높게 써도 된다. 10배에서 30배까지 늘려도 됐다
  - 학습을 빨리 할 수 있다
- dropout을 살짝만 적용해도 된다 (5-10 퍼센트 정도)
  - 논문에서는 쓰지 않아도 된다고 했다
  - Regularization에 많은 수고를 하지 않아도 좋다

## 조금 자세히 보면...
- Input에서의 작은 차이는 NN의 # of layers가 많을수록 파장이 클 것
- Internal covariate shift
  - 매 계층에서의 activation의 distribution이 끊임없기 변하는 것
  - 본래 covariate shift는 $P_{s}(Y|X=x) = P_{t}(Y|X=x)$ for all $x$, but $P_{s}(X) \neq P_{t}(X)$를 말한다 ([A Literature Survey on Domain Adaptation of Statistical Classifiers](http://www.mysmu.edu/faculty/jingjiang/papers/da_survey.pdf) 참조)
    - Source와 target(training과 test)에서 입력(x)가 같으면 출력(y)가 같은 것은 바람직한 것이고, 입력이 달라지는 것이 문제 아니야?
    - 실제로는 $P_{s}$와 $P_{t}$가 같을 수 없는 것이 문제겠지?
- 매 layer마다 입력이 같은 distribution이 되도록 하면 어떨까? 학습이 안정적으로 되지 않을까?
  - 입력이 $N(0, \sigma^{2})$를 따르면 학습이 잘된다는 연구가 이미 있다고 (르쿤 형님의 98년 논문)
  - 전체 feature의 상관관계를 구하고 정규화할 수도 있겠지만 수학적으로 복잡하다고
  - 위에서 말한 르쿤 형님 논문을 보면 feature 간에 상관관계가 있더라도 개별적으로 정규화하면 학습 속도에 도움이 된다고 함
- 각 feature를 개별적으로 $N(0, 1)$을 따르게 변환
  - 변환을 위한 $\mu$와 $\sigma$는 전체 batch에서 가져오는 것이 아니라(역시 수학적으로 부담), mini-batch로부터 추출
- 이걸 그대로 사용하지는 않고 scale & shift 수행
- 이렇게 BN을 통과한 값을 non-linear activation function에 입력으로 준다

### how to train
- 모든 값을 $N(0, 1)$로 정규화 해버리면 non-linearity 효과가 감소하기 때문이라고
- 왜? sigmoid 함수의 모양을 생각해보면, 값이 0 주변에 몰리면 직선처럼 되잖아[3]
  - 그럴싸 하긴 한데... 다른 activation을 사용하면?
- 여튼 $N(0, 1)$ 변환 뒤, 다시 $y = \gamma x + \beta$ 적용
  - $\gamma$와 $\beta$는 학습하도록 함 (논문에서, 두 값이 backprop 가능(미분 가능)함을 간단히 식으로 표현)

### how to infer
- Training 시 얻은 $\mu$와 $\sigma$의 moving average를 대표값으로 사용

### 추가 고려사항
- BN을 적용하면 $N(0,1)$로 변환하기 때문에 bias가 별로 의미가 없을 수 있다고
  - 일리가 있다: bias로 인한 shift가 우선 무시된 후 *scale & shift* 단계에서 $\beta$로 shift될 것
- CNN에서는 filter(channel) 마다 BN을 해주는 형태로 하면 된다고
- Layer Norm에서 이야기한 것처럼 BN을 RNN에 적용하는 건 좀 어려울 것 같긴 함
  - 논문 말미에서는 RNN에도 적용해보고 싶다고 이야기하긴 했다...

## 믿거나 말거나
- 각 계층이 더 독립적인 특징을 표현할 수 있게 된다고

In [1]:
import tensorflow as tf

tf.enable_eager_execution()
keras = tf.keras

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [3]:
def get_model(with_bn:bool=False):
    inp = keras.layers.Input(shape=(28, 28,))
    x = keras.layers.Flatten()(inp)
    x = keras.layers.Dense(1024)(x)
    if with_bn:
        x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(tf.nn.relu)(x)
    x = keras.layers.Dense(1024)(x)
    if with_bn:
        x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(tf.nn.relu)(x)
    x = keras.layers.Dense(10)(x)
    out = keras.layers.Activation(tf.nn.softmax)(x)

    return keras.models.Model(inputs=inp, outputs=out)

In [8]:
model = get_model(with_bn=False)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, shuffle=True, batch_size=128)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 46us/sample - loss: 0.4634 - acc: 0.8334 - val_loss: 0.3957 - val_acc: 0.8529
Epoch 2/20
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3471 - acc: 0.8717 - val_loss: 0.3594 - val_acc: 0.8675
Epoch 3/20
60000/60000 [==============================] - 2s 40us/sample - loss: 0.3116 - acc: 0.8846 - val_loss: 0.3609 - val_acc: 0.8661
Epoch 4/20
60000/60000 [==============================] - 2s 38us/sample - loss: 0.2882 - acc: 0.8923 - val_loss: 0.3372 - val_acc: 0.8795
Epoch 5/20
60000/60000 [==============================] - 2s 39us/sample - loss: 0.2695 - acc: 0.8982 - val_loss: 0.3394 - val_acc: 0.8757
Epoch 6/20
60000/60000 [==============================] - 2s 39us/sample - loss: 0.2564 - acc: 0.9032 - val_loss: 0.3414 - val_acc: 0.8817
Epoch 7/20
60000/60000 [==============================] - 2s 38us/sample - loss: 0.2419 - acc: 0.9079 - val_loss: 0.

In [9]:
model = get_model(with_bn=True)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, shuffle=True, batch_size=128)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 60us/sample - loss: 0.4281 - acc: 0.8450 - val_loss: 0.4167 - val_acc: 0.8519
Epoch 2/20
60000/60000 [==============================] - 3s 49us/sample - loss: 0.3224 - acc: 0.8814 - val_loss: 0.3620 - val_acc: 0.8666
Epoch 3/20
60000/60000 [==============================] - 3s 49us/sample - loss: 0.2811 - acc: 0.8960 - val_loss: 0.3457 - val_acc: 0.8731
Epoch 4/20
60000/60000 [==============================] - 3s 49us/sample - loss: 0.2552 - acc: 0.9043 - val_loss: 0.3567 - val_acc: 0.8757
Epoch 5/20
60000/60000 [==============================] - 3s 49us/sample - loss: 0.2301 - acc: 0.9130 - val_loss: 0.3471 - val_acc: 0.8806
Epoch 6/20
60000/60000 [==============================] - 3s 50us/sample - loss: 0.2161 - acc: 0.9179 - val_loss: 0.3861 - val_acc: 0.8668
Epoch 7/20
60000/60000 [==============================] - 3s 50us/sample - loss: 0.1974 - acc: 0.9244 - val_loss: 0.

In [10]:
bn1 = model.get_layer(index=3)
bn1.get_config()

{'axis': [1],
 'beta_constraint': None,
 'beta_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}},
 'beta_regularizer': None,
 'center': True,
 'dtype': 'float32',
 'epsilon': 0.001,
 'gamma_constraint': None,
 'gamma_initializer': {'class_name': 'Ones', 'config': {'dtype': 'float32'}},
 'gamma_regularizer': None,
 'momentum': 0.99,
 'moving_mean_initializer': {'class_name': 'Zeros',
  'config': {'dtype': 'float32'}},
 'moving_variance_initializer': {'class_name': 'Ones',
  'config': {'dtype': 'float32'}},
 'name': 'batch_normalization_v1_2',
 'scale': True,
 'trainable': True}

In [11]:
bn = keras.models.Model(inputs=model.input, outputs=bn1.output)
bn.predict(x_test)

array([[-1.1696905 , -0.16317093, -1.6329446 , ..., -0.01780403,
        -0.59661084, -0.27380526],
       [-0.35027122, -1.3151393 , -0.8904851 , ..., -1.6451683 ,
        -2.409012  , -1.4611003 ],
       [-0.9416815 , -1.1546845 ,  0.05267343, ...,  0.47187477,
        -0.11088882, -2.2367082 ],
       ...,
       [-1.9248763 , -1.0279324 , -0.28468928, ..., -0.44335788,
        -0.36089683,  1.8417222 ],
       [-0.56644845, -0.18640304, -0.0636247 , ...,  0.08721903,
         0.17327638, -0.86660886],
       [-1.8171611 ,  0.76698005, -0.80402786, ..., -0.16809675,
        -0.2501657 ,  0.3068874 ]], dtype=float32)

In [12]:
bn1.beta

<tf.Variable 'batch_normalization_v1_2/beta:0' shape=(1024,) dtype=float32, numpy=
array([-0.3541091 , -0.28960064, -0.26902372, ..., -0.3008392 ,
       -0.3634126 , -0.22003344], dtype=float32)>

In [13]:
bn1.gamma

<tf.Variable 'batch_normalization_v1_2/gamma:0' shape=(1024,) dtype=float32, numpy=
array([0.9013632 , 1.0713109 , 0.8562308 , ..., 0.95761645, 0.80460775,
       0.89801055], dtype=float32)>

In [14]:
import numpy as np

In [15]:
(_11 - _12) / _13

<tf.Tensor: id=362568, shape=(10000, 1024), dtype=float32, numpy=
array([[-0.90483105,  0.11801402, -1.592936  , ...,  0.29556212,
        -0.2898285 , -0.05987883],
       [ 0.00425787, -0.9572746 , -0.7258106 , ..., -1.4038284 ,
        -2.5423563 , -1.3820182 ],
       [-0.65187085, -0.80750036,  0.3757131 , ...,  0.80691385,
         0.31384706, -2.245714  ],
       ...,
       [-1.7426573 , -0.6891854 , -0.01829596, ..., -0.1488265 ,
         0.0031267 ,  2.2959146 ],
       [-0.23557578,  0.09632835,  0.23988743, ...,  0.40523344,
         0.6670194 , -0.72000873],
       [-1.6231549 ,  0.9862503 , -0.6248364 , ...,  0.13861755,
         0.14074795,  0.5867646 ]], dtype=float32)>

In [17]:
np.mean(np.mean(_15, axis=0))

0.015220008

In [18]:
np.mean(np.std(_15, axis=0))

1.0081763

In [19]:
bn1.moving_mean

<tf.Variable 'batch_normalization_v1_2/moving_mean:0' shape=(1024,) dtype=float32, numpy=
array([ 5.7244453 , -1.7852176 , -0.7978444 , ..., -0.79731953,
       -0.26770753, -1.8675565 ], dtype=float32)>

In [20]:
bn1.moving_variance

<tf.Variable 'batch_normalization_v1_2/moving_variance:0' shape=(1024,) dtype=float32, numpy=
array([5.2919035, 2.3144853, 4.5314775, ..., 1.704213 , 1.2282873,
       1.8428802], dtype=float32)>

## 반전

- [How Does Batch Normalization Help Optimization?](https://papers.nips.cc/paper/7515-how-does-batch-normalization-help-optimization)
- NIPS 2018
- Shibani Santurkar et al., MIT

### 세렌디피티

> the positive impact of BatchNorm on training might be somewhat serendipitous

### Internal covariate shift는 줄지 않아

- 실제로 histogram을 그려보면 BN의 출력이 그다지 가우시안을 따르지 않는다
- 바닐라 네트워크보다 더 많은 covariate shift가 발생하기도 한다

### 그러면 왜 잘되는거지?

- loss landscape를 아주 안정적으로 만들기 때문이라고
  - batchNorm이 아니어도 그냥 L1, L2, L-inf norm으로 normalization을 해도 동일한 효과를 얻었다고
- BN의 장점은 *학습이 안정적이기 때문에* 초기값 등의 하이퍼 파라미터에 민감하지 않아도 되는 것에 있다는 것
  - 추가로 batchNorm이 학습을 more flat minima에 converge하게 만들어줌으로써 generalization에 더 좋다는 이야기도 있으나, 여기에 대한 증명은 없어보인다